In [ ]:
import skimage
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

In [ ]:
data_dir = Path('/Users/asa.barthmaron/Desktop/1__2023-02-27T21_51_31-Measurement 5/Images')
# test_dir = Path('/Users/asa.barthmaron/Desktop')

In [ ]:
fov_prefixes = np.sort(np.unique([f.name[:9] for f in data_dir.glob('*.tiff')]))
channels = np.sort(np.unique([f.name[13:16] for f in data_dir.glob('*.tiff')]))

In [ ]:
for prefix in fov_prefixes:
    for ch in channels:

        z_section_files = [x.name for x in 
                           data_dir.glob(prefix + '*' + ch +'*')]
        z_section_files = np.sort(z_section_files)

        img_stack = np.zeros((z_section_files.shape[0], 1080, 1080))
        img_stack = img_stack.astype('uint16')

        for i_f, f in enumerate(z_section_files):
            tif = skimage.io.imread(data_dir / f)
            img_stack[i_f, :, :] = tif

        mip = img_stack.max(axis=0)
        # skimage.io.imsave(data_dir.parents[0] / 'MIPs' / z_section_files[0], mip)
        skimage.io.imsave(data_dir / z_section_files[0], mip)
        for file in z_section_files[1:]:
            os.remove(data_dir / file)

In [ ]:
data_dir = Path('/Volumes/Lab$/ABM/2023-02-14_DIV11_NGN2_HOECHST_TDP-43_MAP2/DIV11__2023-02-15T11_31_26-Measurement 1/MIPs')
fov_prefixes = np.sort(np.unique([f.name[:9] for f in data_dir.glob('*.tiff')]))

In [ ]:
# channels = ['405', '488', '561', '647']
channels = ['ch3', 'ch1']

for prefix in fov_prefixes[30:]:
    img_stack = np.zeros((2, 1080, 1080))
    img_stack = img_stack.astype('uint16')
    for i_ch, ch in enumerate(channels):

        file = [f.name for f in data_dir.glob(prefix + '*' + ch +'*')][0]
        tif = skimage.io.imread(data_dir / file)
        tif = tif - tif.mean()
        tif = tif / tif.std()
        tif = tif - tif.min()
        tif = tif * 2e3
        # tif = tif - tif.min()
        # tif = tif / tif.max()
        img_stack[i_ch, :, :] = tif

    tifffile.imwrite(data_dir / 'cellpose' / file, 
                        img_stack, metadata={'axes': 'CYX'}, 
                        imagej=True, compression=None, dtype='uint16')
        # skimage.io.imsave(mip_dir / (prefix + ch + '.tif'), mip)